# This notebook is to test a single batch run in ADAM

In [1]:
from adam import Batch
from adam import Batches
from adam import BatchRunManager
from adam import PropagationParams
from adam import OpmParams
from adam import ConfigManager
from adam import Projects
from adam import RestRequests
from adam import AuthenticatingRestProxy
from adam import AdamProcessingService
from adam import BatchPropagationResults
import time
import os

This sets up authenticated access to the server. It needs to be done before pretty much everything you want to do with ADAM.

In [2]:
# Reads your config from a file in current directory. For instructions on setting this up, see config_demo notebook.
config = ConfigManager().get_config('localdev')
auth_rest = AuthenticatingRestProxy(RestRequests(config['url']), config['token'])
aps = AdamProcessingService(auth_rest)

## Example Inputs

In [3]:
# Keplerian elements state vector
keplerian_elements = {
    'semi_major_axis_km': 448793612,
    'eccentricity': 0.1,
    'inclination_deg': 90,
    'ra_of_asc_node_deg': 91,
    'arg_of_pericenter_deg': 92,
    'true_anomaly_deg': 93,
    'gm': 132712440041.9394
}

# Uncertainties on elements
keplerian_sigma = {
    'semi_major_axis': 100,
    'eccentricity': 0.001,
    'inclination': 1,
    'ra_of_asc_node': 2,
    'arg_of_pericenter': 3,
    'true_anomaly': 4,
}


### Set Parameters

Commented parameters are optional. Uncomment to use.

In [4]:
propagation_params = PropagationParams({
    'start_time': '2017-10-04T00:00:00Z',  # propagation start time in ISO format
    'end_time': '2017-10-11T00:00:00Z',  # propagation end time in ISO format

    'keplerianSigma': keplerian_sigma,
    'monteCarloDraws': 100,
    'propagationType': 'MONTE_CARLO'
})

opm_params = OpmParams({
    'epoch': '2017-10-04T00:00:00Z',
    'keplerian_elements': keplerian_elements,
})


### Submit and Run Propagation

In [5]:
batch_run = aps.execute_batch_propagation(config['workspace'], propagation_params, opm_params)
# batch_run = BatchPropagationResults.fromRESTwithRawIds(auth_rest, '0dc1e8b0-4f92-46ad-8838-c9e9eca6935c', '093a2424-9dfb-4cae-88bf-a9077659e8ca')
print(batch_run)

{'job_uuid': '093a2424-9dfb-4cae-88bf-a9077659e8ca', 'results': None}


### Get Status

In [6]:
print(batch_run.check_status())

{'status': 'COMPLETED'}


### Get Number of Runs

In [7]:
# Get ephemeris of specified part
runs_count = batch_run.get_result_ephemeris_count()
print(f'# Runs: {runs_count}')

# Runs: 100


### Get Ephemeris for specific run

In [8]:
eph = batch_run.get_result_ephemeris(2)
print(eph)

stk.v.11.0
BEGIN Ephemeris
ScenarioEpoch 23 May 2020 12:00:00.000000
CentralBody SUN
CoordinateSystem ICRF
InterpolationMethod HERMITE
InterpolationOrder 5
NumberOfEphemerisPoints 366

EphemerisTimePosVel
0.000000000000e+00 -3.351566612068e+11 -6.313014871554e+10 1.160353265453e+11 3.643749455537e+03 -2.060949361099e+04 1.744582173580e+03
8.640000000000e+05 -3.316552157328e+11 -8.086369181877e+10 1.174194509008e+11 4.459170633125e+03 -2.043333010135e+04 1.459143067847e+03
1.728000000000e+06 -3.274555157897e+11 -9.842694191337e+10 1.185564061086e+11 5.259661161165e+03 -2.021550289054e+04 1.172593550880e+03
2.592000000000e+06 -3.225715232727e+11 -1.157844659105e+11 1.194456506462e+11 6.042788992580e+03 -1.995735269839e+04 8.858977176723e+02
3.456000000000e+06 -3.170192319694e+11 -1.329020752591e+11 1.200874663637e+11 6.806285431173e+03 -1.966041625130e+04 5.999959415574e+02
4.320000000000e+06 -3.108165196866e+11 -1.497469860941e+11 1.204829342478e+11 7.548060330300e+03 -1.932640464328e+0

### Get ending state vector

In [11]:
# Get the end state vector (uncomment to use)
end_state_vector = batch_run.get_final_positions()['hit']['finalPosition'][2]
print("State vector at the end of propagation:")
print(end_state_vector)

State vector at the end of propagation:
{'x': -3163.9565918747803, 'y': -430.3596367332733, 'z': -5521.304271507949, 'epoch': None, 'x_dot': 0, 'y_dot': 0, 'z_dot': 0, 'comments': []}
